<center>
    
# [Методы искусственного интеллекта в анализе данных](http://rairi.ru/wiki/index.php/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D0%B8%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D0%B0_%D0%B2_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B5_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85)

## <center> Семинар 5
    
***

## План семинара
1. [KNN: выбор числа соседей](#1.-KNN:-выбор-числа-соседей)
    - 1.1. [Метрические методы](#1.1.-Метрические-методы)
    - 1.2. [KNN в Scikit-Learn](#1.2.-KNN-в-Scikit-Learn)
    - 1.3. [Выбор числа соседей](#1.3.-Выбор-числа-соседей)
2. [KNN: выбор метрики](#2.-KNN:-выбор-метрики)
    - 2.1. [Метрика](#2.1.-Метрика)
    - 2.2. [Реализация в Scikit-Learn](#2.2.-Реализация-в-Scikit-Learn)
    - 2.3. [Выбор метрики](#2.3.-Выбор-метрики)
3. [SVM: опорные объекты](#3.-SVM:-опорные-объекты)
    - 3.1. [Метод опорных векторов](#3.1.-Метод-опорных-векторов)
    - 3.2. [Метод опорных векторов в Scikit-Learn](#3.2.-Метод-опорных-векторов-в-Scikit-Learn)
    - 3.3. [Опорные объекты](#3.3.-Опорные-объекты)
4. [SVM: анализ текстов](#4.-SVM:-анализ-текстов)
    - 4.1. [Разреженные данные](#4.1.-Разреженные-данные)
    - 4.2. [Анализ текстов](#4.2.-Анализ-текстов)

## 1. KNN: выбор числа соседей

### 1.1. Метрические методы

Метрические методы основаны на гипотезе компактности, суть которой состоит в том, что объекты с похожими признаковыми описаниями имеют похожие значения целевой переменной. Если эта гипотеза верна, то строить прогноз для нового объекта можно на основе близких к нему объектов из обучающей выборки — например, путем усреднения их ответов (для регрессии) или путем выбора наиболее популярного среди них класса (для классификации). Методы такого типа и называются метрическими. Они имеют несколько особенностей:

- Процедура обучения, по сути, отсутствует — достаточно лишь запомнить все объекты обучающей выборки
- Можно использовать метрику, учитывающую особенности конкретного набора данных — например, наличие категориальных (номинальных) признаков
- При правильном выборе метрики и достаточном размере обучающей выборки метрические алгоритмы показывают качество, близкое к оптимальному

Метрические методы чувствительны к масштабу признаков — так, если масштаб одного из признаков существенно превосходит масштабы остальных признаков, то их значения практически не будут влиять на ответы алгоритма. Поэтому важно производить масштабирование признаков. Обычно это делается путем вычитания среднего значения признака и деления на стандартное отклонение.

К метрическим алгоритмам классификации относятся:

- [Метод ближайших соседей](http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%B1%D0%BB%D0%B8%D0%B6%D0%B0%D0%B9%D1%88%D0%B8%D1%85_%D1%81%D0%BE%D1%81%D0%B5%D0%B4%D0%B5%D0%B9)
- [Метод потенциальных функций](http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BF%D0%BE%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B9)
- [Метод парзеновского окна](http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BF%D0%B0%D1%80%D0%B7%D0%B5%D0%BD%D0%BE%D0%B2%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D0%BE%D0%BA%D0%BD%D0%B0)
- [Метод дробящихся эталонов](http://synset.com/ai/ru/recognition/Recognition_01_Etalons.html)
- и др.

На данном семенаре мы подробнее познакомимся с **методом ближайших соседей** (k-nearest neighbors)

### 1.2. KNN в Scikit-Learn

**Метод k ближайших соседей** реализован в классе [`sklearn.neighbors.KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). Основным параметром является `n_neighbors`, который задает число соседей для построения прогноза.

Вам понадобится производить кросс-валидацию по блокам. Мы уже говорили о кросс-валидации на прошлых занятиях, поэтому здесь только кратко повторим основные моменты.

**Кросс-валидация** заключается в разделении выборки на `m` непересекающихся блоков примерно одинакового размера, после чего выполняется `m` шагов. На `i`-м шаге `i`-й блок выступает в качестве тестовой выборки, объединение всех остальных блоков — в качестве обучающей выборки. Соответственно, на каждом шаге алгоритм обучается на некоторой обучающей выборке, после чего вычисляется его качество на тестовой выборке. После выполнения `m` шагов мы получаем `m` показателей качества, усреднение которых и дает оценку кросс-валидации. Подробнее про кросс-валидацию можно почитать на Википедии ([на английском](https://en.wikipedia.org/wiki/Cross-validation_(statistics))) или в документации [scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html).

Технически **кросс-валидация** проводится в два этапа:

1. Создается генератор разбиений `sklearn.model_selection.KFold`, который задает набор разбиений на обучение и валидацию. Число блоков в кросс-валидации определяется параметром `n_splits`. Обратите внимание, что порядок следования объектов в выборке может быть неслучайным, это может привести к смещенности кросс-валидационной оценки. Чтобы устранить такой эффект, объекты выборки случайно перемешивают перед разбиением на блоки. Для перемешивания достаточно передать генератору `KFold` параметр `shuffle=True`.

2. Вычислить качество на всех разбиениях можно при помощи функции `sklearn.model_selection.cross_val_score`. В качестве параметра `estimator` передается классификатор, в качестве параметра `cv` — генератор разбиений с предыдущего шага. С помощью параметра `scoring` можно задавать меру качества, по умолчанию в задачах классификации используется доля верных ответов (`accuracy`). Результатом является массив, значения которого нужно усреднить.

Приведение признаков к одному масштабу можно делать с помощью функции `sklearn.preprocessing.scale`, которой на вход необходимо подать матрицу признаков и получить масштабированную матрицу, в которой каждый столбец имеет нулевое среднее значение и единичное стандартное отклонение.

### 1.3. Выбор числа соседей

В этом задании вам нужно подобрать оптимальное значение `k` для алгоритма **kNN**. Будем использовать набор данных **Wine**, где требуется предсказать сорт винограда, из которого изготовлено вино, используя результаты химических анализов.

Выполните следующие шаги:

**1.3.1.** Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data (файл также лежит в архиве)

**1.3.2** Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names в архиве)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale

In [ ]:
data_raw = pd.read_csv('wine.data', names = ['Target','Alcohol', 'Malic acid','Ash', 'Alcalinity','Magnesium','TPhenols','Flavanoids','Nonflav','Proan','Color','Hue','OD','Proline'])

In [ ]:
data_raw.head()

In [ ]:
X = data_raw.iloc[:,1:]
y = data_raw['Target']

**1.3.3.** Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (`shuffle=True`). Для воспроизводимости результата, создавайте генератор `KFold` с фиксированным параметром `random_state=42`. В качестве меры качества используйте долю верных ответов (`accuracy`).

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

**1.3.4** Найдите точность классификации на кросс-валидации для метода k ближайших соседей (`sklearn.neighbors.KNeighborsClassifier`), при `k` от `1` до `50`. При каком `k` получилось оптимальное качество? Чему оно равно (число в интервале от `0` до `1`)?

In [ ]:
scores = []

for i in range(1,51):
    knn = KNeighborsClassifier(n_neighbors=i)
    current_score = np.mean(cross_val_score(knn, X, y, cv=kf, scoring='accuracy'))
    scores.append(current_score)
    print(i, current_score)

In [ ]:
optimal_k = np.array(scores).argmax()
optimal_score = np.array(scores).max()

print(optimal_k + 1, optimal_score, sep='\n')

**Ответ для самопроверки:**
1. При каком `k` получилось оптимальное качество?: **1**
2. Чему оно равно (число в интервале от `0` до `1`)?: **0.73**

**1.3.5.** Произведите масштабирование признаков с помощью функции `sklearn.preprocessing.scale`. Снова найдите оптимальное k на кросс-валидации.

In [ ]:
X_norm = scale(X)

In [ ]:
scores_norm = []

for i in range(1,51):
    knn = KNeighborsClassifier(n_neighbors=i)
    current_score_norm = np.mean(cross_val_score(knn, X_norm, y, cv=kf, scoring='accuracy'))
    scores_norm.append(current_score_norm)
    print(i, current_score_norm)

**1.3.6.** Какое значение `k` получилось оптимальным после приведения признаков к одному масштабу? Помогло ли масштабирование признаков?

In [ ]:
optimal_k_norm = np.array(scores_norm).argmax()
optimal_score_norm = np.array(scores_norm).max()

print(optimal_k_norm + 1, optimal_score_norm, sep='\n')

**Ответ для самопроверки:**
1. Какое значение `k` получилось оптимальным после приведения признаков к одному масштабу?: **29**
2. Какое при этом получилось качество?: **0.98**

## 2. KNN: выбор метрики

### 2.1. Метрика

Главным параметром любого метрического алгоритма является **функция расстояния** (или метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, **евклидову метрику**), но гораздо более эффективным вариантом является подбор метрики под конкретную задачу. Один из подходов — использование той же евклидовой метрики, но с весами: каждой координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние. Веса настраиваются с целью оптимизации качества на отложенной выборке. Другой подход, о котором и пойдет речь на семинаре — выбор метрики из некоторого класса метрик.

Мы возьмем за основу **метрику Минковского**:

$$ \rho_p (x, z) = \Bigg(\sum_{j=1}^{d} \left|x_j - z_j \right|^{p}\Bigg)^{\frac{1}{p}}$$

Параметром **метрики Минковского** является число **p**, которое мы и будем настраивать.

### 2.2. Реализация в Scikit-Learn

Нам понадобится решать задачу регрессии с помощью метода **k ближайших соседей** — воспользуйтесь для этого классом [`sklearn.neighbors.KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html). Метрика задается с помощью параметра `metric`, нас будет интересовать значение `’minkowski’`. Параметр метрики Минковского задается с помощью параметра `p` данного класса.

### 2.3. Выбор метрики

Мы будем использовать в данном задании набор данных **Boston**, где нужно предсказать стоимость жилья на основе различных характеристик расположения (загрязненность воздуха, близость к дорогам и т.д.). Подробнее о признаках можно почитать по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/housing/. Данный можно загружать непосредственно из `sklearn` или воспользоваться файлами из архива.

**2.3.1.** Загрузите выборку `Boston` с помощью функции `sklearn.datasets.load_boston()`. Результатом вызова данной функции является объект, у которого признаки записаны в поле `data`, а целевой вектор — в поле `target`.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
boston =  load_boston()

In [ ]:
X = boston.data
y = boston.target

**2.3.2.** Приведите признаки в выборке к одному масштабу при помощи функции `sklearn.preprocessing.scale`.

In [ ]:
X_norm = scale(X)

**2.3.3.** Переберите разные варианты параметра метрики `p` по сетке от `1` до `10` с таким шагом, чтобы всего было протестировано `200` вариантов (используйте функцию `numpy.linspace`). Используйте `KNeighborsRegressor` с `n_neighbors=5` и `weights='distance`' — данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества используйте среднеквадратичную ошибку (параметр `scoring='neg_mean_squared_error'` у `cross_val_score`). Качество оценивайте с помощью кросс-валидации по `5` блокам с `random_state = 42`, не забудьте включить перемешивание выборки (`shuffle=True`).

In [ ]:
eval_arr = np.linspace(1, 10, num=200)

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [ ]:
scores = []

for i in eval_arr:
    neigh = KNeighborsRegressor(n_neighbors=5, weights='distance', metric='minkowski', p=i)
    current_score = np.mean(cross_val_score(neigh, X, y, cv=kf, scoring='neg_mean_squared_error'))
    scores.append(current_score)
    print('p: %0.4f, score: %0.4f' % (i, current_score))

**2.3.4.** Определите, при каком `p` качество на кросс-валидации оказалось оптимальным. Обратите внимание, что `cross_val_score` возвращает массив показателей качества по блокам; необходимо максимизировать среднее этих показателей. Это значение параметра и будет ответом.

In [ ]:
optimal_p = eval_arr[np.array(scores).argmax()]

print(optimal_p)

**Ответ для самопроверки:**
1. `p`: **1**


## 3. SVM: опорные объекты

### 3.1. Метод опорных векторов 

**Метод опорных векторов** (**Support Vector Machine**, **SVM**) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Метод опорных векторов имеет еще одну особенность. Если преобразовать его оптимизационную задачу, то окажется, что итоговый классификатор можно представить как взвешенную сумму скалярных произведений данного объекта на объекты обучающей выборки:

$$a(x) = \sum_{i=1}^{l} \lambda_i y_i \left\langle {x, x_i} \right\rangle - w_0$$

По сути, алгоритм делает предсказания на основе сходства нового объекта с объектами обучающей выборки. При этом, как правило, далеко не все коэффициенты оказываются ненулевыми. Это означает, что классификация делается на основе сходства лишь с частью обучающих объектов. Такие объекты называются **_опорными_**.

### 3.2. Метод опорных векторов в Scikit-Learn

**Метод опорных векторов** реализован в классе [`sklearn.svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

Основными параметрами этого класса являются коэффициент `С` и тип ядра `kernel`. Мы будем использовать линейное ядро — для этого нужно задать значение параметра `kernel='linear'`.
Индексы опорных объектов обученного классификатора хранятся в поле `support_`.

### 3.3. Опорные объекты

**3.3.1.** Загрузите выборку из файла ёsvm-data.csvё. В нем записана двумерная выборка (целевая переменная указана в первом столбце, признаки — во втором и третьем).

In [ ]:
from sklearn.svm import SVC

In [ ]:
data= pd.read_csv('svm-data.csv', names=['Target', 'x1','x2'])

In [ ]:
data.head()

In [ ]:
X = data.iloc[:,1:]
y = data['Target']

**3.3.2.** Обучите классификатор с линейным ядром, параметром `C = 100000` и `random_state=241`. Такое значение параметра нужно использовать, чтобы убедиться, что **SVM** работает с выборкой как с линейно разделимой. При более низких значениях параметра алгоритм будет настраиваться с учетом слагаемого в функционале, штрафующего за маленькие отступы, из-за чего результат может не совпасть с решением классической задачи **SVM** для _линейно разделимой выборки_.

In [ ]:
clf = SVC(C=100000, random_state=241, kernel='linear')

In [ ]:
clf.fit(X, y)

**3.3.3.** Найдите номера объектов, которые являются опорными (нумерация с единицы). Они будут являться ответом.

In [ ]:
clf.support_

**Ответ для самопроверки:**
1. Номера опорных объектов: **4 5 10**

## 4. SVM: анализ текстов

### 4.1. Разреженные данные

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на **разреженных данных**. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью **"мешка слов"** — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а [**TF-IDF**](https://ru.wikipedia.org/wiki/TF-IDF). Это показатель, который равен произведению двух чисел: **TF** (**term frequency**) и **IDF** (**inverse document frequency**). Первое равно отношению числа вхождений слова в документ к общей длине документа. Второе число зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, **TF-IDF** будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

Как уже было сказано, линейные методы часто применяются для решения различных задач анализа текстов. Мы применим **метод опорных векторов** для определения того, к какой из тематик относится новость: атеизм или космос.

### 4.2. Анализ текстов

Для начала загрузим данные. Мы воспользуемся одним из датасетов, доступных в **scikit-learn'е** — `20 newsgroups`. Для этого нужно воспользоваться модулем `datasets`:

In [ ]:
# from sklearn import datasets

# newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

После выполнения этого кода массив с текстами будет находиться в поле `newsgroups.data`, номер класса — в поле `newsgroups.target`.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление `TF-IDF`. В **Scikit-Learn** это реализовано в классе [`sklearn.feature_extraction.text.TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Преобразование обучающей выборки нужно делать с помощью функции `fit_transform`, тестовой — с помощью `transform`.

Реализация SVM-классификатора находится в классе [`sklearn.svm.SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). Веса каждого признака у обученного классификатора хранятся в поле `coef_`. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом `get_feature_names()` у `TfidfVectorizer`:

In [ ]:
# feature_mapping = vectorizer.get_feature_names()
# print(feature_mapping[i])

Подбор параметров удобно делать с помощью класса [`sklearn.model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Первым аргументом в `GridSearchCV` передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (`dict`), задающий сетку параметров для перебора.

**4.2.1** Загрузите объекты из новостного датасета `20 newsgroups`, относящиеся к категориям "космос" и "атеизм". Обратите внимание, что загрузка данных может занять несколько минут

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

In [ ]:
X = newsgroups.data
y = newsgroups.target

**4.2.2** Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что предлагается вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X_tf = vectorizer.fit_transform(X)

**4.2.3.** Подберите минимальный лучший параметр C из множества $[10^{-5}, 10^{-4}, ... 10^4, 10^5]$ для SVM с линейным ядром (`kernel='linear'`) при помощи кросс-валидации по `5` блокам. Укажите параметр `random_state=241` и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (`accuracy`).

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=241)

In [ ]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}

clf = SVC(kernel='linear', random_state=241)

gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_tf, y)

In [ ]:
for i in zip(gs.cv_results_['mean_test_score'], gs.cv_results_['params']):
    print('Score: %0.7f, C: %s' % (i[0], i[1]['C']))

**4.2.4.** Обучите SVM по всей выборке с оптимальным параметром `C`, найденным на предыдущем шаге.

In [ ]:
clf_opt = SVC(kernel='linear', random_state=241, C=1.0)
clf_opt.fit(X_tf,y)

**4.2.5.** Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле `coef_` у `svm.SVC`). Они являются ответом.

In [ ]:
top10idx = np.array(clf_opt.coef_.indices)[np.abs(np.array(clf_opt.coef_.data)).argsort()[-10:]]
print(top10idx)

In [ ]:
feature_mapping = vectorizer.get_feature_names()

In [ ]:
for iter in top10idx:
    print(feature_mapping[iter])

**Ответ для самопроверки:**
1. **atheism**
2. **atheists**
3. **bible** 
4. **god** 
5. **keith** 
6. **moon** 
7. **religion** 
8. **sci** 
9. **sky** 
10. **space**